## NLTK POS Tagging + sklearn TF-IDF + Gensim Topic Modeling
A new way forward

In [35]:
from sklearn.datasets.base import Bunch
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

import re
from stop_words import get_stop_words

import numpy as np
import pandas as pd

import time
import numpy as np
import matplotlib.pyplot as plt

# Some NLTK specifics
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk import RegexpTokenizer


import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim

from pprint import pprint

import codecs
import os
import time

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/goodgame/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/goodgame/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/goodgame/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/goodgame/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/goodgame/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/goodgame/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#CONNECTING TO THE DATASET
CORPUS_ROOT = "/Users/goodgame/desktop/Shift/match/reqs/jobs_text_meat_only/"

def load_data(root=CORPUS_ROOT):
    """
    Loads the text data into memory using the bundle dataset structure.
    Note that on larger corpora, memory safe CorpusReaders should be used.
    """

    # Open the README and store
    with open(os.path.join(root, 'README'), 'r') as readme:
        DESCR = readme.read()

    # Iterate through all the categories
    # Read the HTML into the data and store the category in target
    data      = []
    target    = []
    filenames = []

    for category in os.listdir(root):
        if category == "README": continue # Skip the README
        if category == ".DS_Store": continue # Skip the .DS_Store file
        for doc in os.listdir(os.path.join(root, category)):
            if doc == ".DS_Store": continue
            fname = os.path.join(root, category, doc)

            # Store information about document
            filenames.append(fname)
            target.append(category)
            with codecs.open(fname, 'r', 'ISO-8859-1') as f:
                data.append(f.read())
            # Read data and store in data list
            # with open(fname, 'r') as f:
            #     data.append(f.read())

    return Bunch(
        data=data,
        target=target,
        filenames=filenames,
        target_names=frozenset(target),
        DESCR=DESCR,
    )

dataset = load_data()

#print out the readme file
print(dataset.DESCR)
#Remember to create a README file and place it inside your CORPUS ROOT directory if you haven't already done so.

#print the number of records in the dataset
print("The number of instances is ", len(dataset.data), "\n")

This is a corpus of job descriptions.

They have been stripped of any boilerplate text, leaving behind only the text specific to a given job.

In other words, lines like "Here at Uber, we believe in..." have been removed.

Additionally, lines "introducing" meaty job description text have been removed. Here are some examples:
- "About you:"
- "Required Skills"
- "What You'll Need"

The number of instances is  1347 



In [5]:
job_str = '''Data Scientist/Machine Learning Engineer, Adaptive Authentication
Position Description:

This is an opportunity to join our fast-growing Adaptive Authentication team to develop cutting-edge risk-based adaptive authentication policies. We are looking for a Data Scientist/Machine Learning Engineer to build large-scale distributed systems while using machine learning to solve business problems. The ideal candidate has experience building models from complex systems, developing enterprise-grade software in an object-oriented language, and experience or knowledge in security, authentication or identity.

Our elite team is fast, innovative and flexible; with a weekly release cycle and individual ownership we expect great things from our engineering and reward them with stimulating new projects and emerging technologies.


Job Duties and Responsibilities:

Build and own models that identify risk associated with anomalous activity in the cloud for authentication
Build Machine Learning pipelines for training and deploying models at scale
Analyze activity data in the cloud for new behavioral patterns
Partner with product leaders to define requirements for building models
Work closely with engineering lead and management to scope and plan engineering efforts
Test-driven development, design and code reviews

Required Skills:

2+ years of Data Science/Machine Learning experience
Skilled in using machine learning algorithms for classification and regression
5+ years of software development experience in an object-oriented language building highly-reliable, mission-critical software
Excellent grasp of software engineering principles
Experience with multi-factor authentication, security, or identity is a plus

Education:

B.S, M.S, or Ph.D. in computer science, data science, machine learning, information retrieval, math or equivalent work experience

 

Okta is an Equal Opportunity Employer'''

In [6]:
stopwords = stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

In [31]:
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

def normalise(word):
    word = word.lower().replace('/','').replace('-','').replace('•','')
    # word = stemmer.stem_word(word) #if we consider stemmer then results comes with stemmed word, but in this case word will not match with comment
    word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword. We can increase the length if we want to consider large phrase"""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term

In [32]:
# combine functions above
def noun_phrases(text):
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')    
    lemmatizer = nltk.WordNetLemmatizer()
    stemmer = nltk.stem.porter.PorterStemmer()
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """
    chunker = nltk.RegexpParser(grammar)
    toks = tokenizer.tokenize(text)
    postoks = nltk.tag.pos_tag(toks)
    tree = chunker.parse(postoks)
    terms = get_terms(tree)
    bad_words = ['opportunity', 'ideal candidate', 'team', 'year', 'knowledge','experience']
    clean_terms = []
    
    for term in terms:
        term = ' '.join(term).replace('\n','').replace(',','').replace('(','')
        term = term.replace(')','')
        term = term.strip()
        if term not in bad_words:
            clean_terms.append(term)
    return clean_terms

<class 'str'>


In [34]:
%%time
parsed_dataset = []
titles = []

def parse_input_docs(doc):
    return ' '.join(noun_phrases(doc))

for item in dataset.data:
    titles.append(item.split("\n",2)[0])
    copy_minus_title = item.split("\n",2)[2]
    parsed_dataset.append(parse_input_docs(copy_minus_title))
print(len(parsed_dataset))
print(titles[0])

1347
Uber 362--Designer---Gurgaon--India---Uber---Uber.Txt
CPU times: user 30 s, sys: 119 ms, total: 30.1 s
Wall time: 30.2 s


In [11]:
# Use Gensim Phrases with POS-tagged JDs
# phrases = gensim.models.Phrases(list_pos_jd)
phrases = gensim.models.Phrases(parsed_dataset)
bigram = gensim.models.phrases.Phraser(phrases)

### Transform POS vectors into TF-IDF space. 
The terms should be more useful than the terms from the entire document.

In [18]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [19]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [20]:
# porter_stemmer = PorterStemmer()

def lemmatizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [wordnet_lemmatizer.lemmatize(word) for word in words]
    return words

In [21]:
stop_words = text.ENGLISH_STOP_WORDS

In [22]:
# TF-IDF transformation in sklearn

pos_vect = TfidfVectorizer(stop_words=stop_words, tokenizer=lemmatizer, ngram_range=(1,2), analyzer='word')  
pos_tfidf = pos_vect.fit_transform(parsed_dataset)
print("\n Here are the dimensions of our two-gram dataset: \n", pos_tfidf.shape, "\n")


 Here are the dimensions of our two-gram dataset: 
 (1347, 94141) 



In [23]:
true_k = 100
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(pos_tfidf)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = pos_vect.get_feature_names()
for i in range(true_k):
    print("\n\nCluster %d:" % i, "\n")
    for ind in order_centroids[i, :40]:
        print(' %s' % terms[ind])

Top terms per cluster:


Cluster 0: 

 campaign
 marketing
 crm
 lifecycle
 email
 communication
 team
 customer
 channel
 sm
 contact strategy
 lifecycle marketing
 regional
 messaging
 marketing team
 exacttarget
 strategy
 familiarity crm
 driver
 execution
 deployment
 segmentation
 experimentation
 team pilot
 campaign oversees
 lifecycle experimentation
 channel execution
 mechanism campaign
 new mechanism
 flow campaign
 team definition
 implementation automated
 oversees channel
 automation input
 track performance
 automated tool
 experimentation collaboration
 input global
 notification
 plan support


Cluster 1: 

 technical
 professional service
 customer
 iam
 p
 technical leadership
 solution
 service
 okta
 specification
 architecture
 iam solution
 engagement manager
 identity
 software development
 assist
 requirement
 member professional
 design technical
 okta customer
 professional
 design
 manager technical
 architecture design
 delivery team
 proficiency
 engageme

 enterprise
 business
 cloud
 success
 selling
 ae
 account executive
 product
 solution
 team
 value
 company row
 row
 relationship
 support
 level
 client
 role
 strategic account
 transformation
 revenue
 incentive
 strategic
 industry
 sale cycle
 experience
 software
 growth
 plan
 business value
 strategy
 application


Cluster 54: 

 content
 copy
 creative
 brand
 content strategist
 copywriter
 strategist
 product
 concept
 ux
 way
 v
 user
 iconic brand
 iconic
 people
 work
 brand voice
 voice
 opinion
 strategist copywriter
 creative concept
 impressive
 localization
 right
 uber
 solid
 writing
 idea
 way people
 affirm
 print
 creative decision
 ux copy
 relevant content
 touch point
 life
 time work
 campaign
 great


Cluster 55: 

 salesforce org
 org
 salesforce
 technology
 nonprofit
 org s
 education
 intern
 customer
 volunteer
 data
 world
 social enterprise
 enterprise
 hand nonprofit
 technology hand
 privacy
 equalizer time
 world tool
 endless circle
 time acc

 multiple schedule
 review logistics


Cluster 91: 

 business
 solution
 integration
 salesforce
 project
 client
 company
 application
 cloud
 technical
 enterprise
 technology
 architect
 requirement
 product
 management
 delivery
 customer
 process
 experience
 architecture
 service
 partner
 ability
 design
 business process
 company row
 row
 com
 development
 functional
 scope
 business partner
 implementation
 team
 ability client
 level
 solution architect
 software
 technical architect


Cluster 92: 

 design
 creative
 print
 brand
 collateral
 designer
 visual
 affirm
 marketing
 digital
 adobe creative
 illustrator
 indesign
 asset
 adobe
 campaign
 photoshop
 illustrator indesign
 visual design
 quality
 conceptual thinker
 production
 creative suite
 conceptual
 design experience
 agency
 photoshop illustrator
 promotion
 design spec
 cloud photoshop
 collateral digital
 asset quality
 spec production
 digital promotion
 experience thorough
 promotion asset
 print collat

### Gensim 
#### (with streaming and new data input type)

Note that there's a memory-friendly way to do this by streaming docs.

https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb

I'm not dealing with it yet because I want a POC first. But we'll need to use that if we build this corpus using all of the jobs in S3.

There are a few places where I'd need to use fixed-memory workarounds.

In [27]:
print(parsed_dataset[0])

indiasa creative team designer uber brand brand story quality marketing communication product experience fast early stage role passion creative solution global scale conceive uber storytelling system consult provide resource guideline template colleague thoughtful brand expression source inspiration partnership global team idea and work producer engineer marketer others brand portfolio relevant experience wide range project brand content creative work brand guideline execution web print product packaging problem solver skill user strategic thinker excellent business acumen passion quality word solution pursuit new creative work enthusiasm new opportunity boundary non traditional approach ownership sizeable part project much oversight ambiguity change direction style part organization proper use brand direction effective communication skill design decision work functional team executive organization skill time work


In [12]:
documents = parsed_dataset

In [13]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [14]:
!rm /tmp/parsed.mm

rm: /tmp/parsed.mm: No such file or directory


In [15]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.save('/tmp/parsed.dict')  # store the dictionary, for future reference

In [60]:
# Look at token IDs
# print(dictionary.token2id)

In [16]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # All three words appear in the dictionary

[(189, 1), (245, 1), (1712, 1)]


In [17]:
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize('/tmp/parsed.mm', corpus)  # store to disk, for later use

In [18]:
if (os.path.exists("/tmp/parsed.dict")):
    dictionary = gensim.corpora.Dictionary.load('/tmp/parsed.dict')
    corpus = gensim.corpora.MmCorpus('/tmp/parsed.mm')
    print("Used saved dictionary and corpus")
else:
    print("Please run first tutorial to generate data set")

Used saved dictionary and corpus


In [19]:
print(dictionary[0])
print(dictionary[1])
print(dictionary[2])

acumen
ambiguity
approach


In [20]:
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [21]:
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=500)

In [67]:
# hdp = gensim.models.HdpModel(corpus, id2word=dictionary)

In [22]:
print(job_str)

Data Scientist/Machine Learning Engineer, Adaptive Authentication
Position Description:

This is an opportunity to join our fast-growing Adaptive Authentication team to develop cutting-edge risk-based adaptive authentication policies. We are looking for a Data Scientist/Machine Learning Engineer to build large-scale distributed systems while using machine learning to solve business problems. The ideal candidate has experience building models from complex systems, developing enterprise-grade software in an object-oriented language, and experience or knowledge in security, authentication or identity.

Our elite team is fast, innovative and flexible; with a weekly release cycle and individual ownership we expect great things from our engineering and reward them with stimulating new projects and emerging technologies.


Job Duties and Responsibilities:

Build and own models that identify risk associated with anomalous activity in the cloud for authentication
Build Machine Learning pipeline

In [23]:
doc = parse_input_docs(job_str)
print(doc, "\n\n")
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

data scientist machine learning engineer adaptive authentication position description fast growing adaptive authentication team edge risk adaptive authentication policy data scientist machine learning engineer large scale system machine business problem experience building model complex system enterprise grade software object language security authentication identity elite team weekly release cycle individual ownership great thing engineering new project technology job duty responsibility build model risk anomalous activity cloud authentication build machine learning pipeline training deploying model scale analyze activity data cloud new behavioral pattern partner product leader requirement building model work engineering lead management engineering effort test driven development design code review skill data science machine learning experience machine algorithm classification regression software development experience object language reliable mission critical software excellent grasp 

In [24]:
index = gensim.similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [25]:
!rm /tmp/parsed.index
index.save('/tmp/parsed.index')

rm: /tmp/parsed.index: No such file or directory


In [26]:
sims = index[vec_lsi]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
for item in sims_sorted[:5]:
    print(titles[item[0]],"\n\tIndex:",item[0],"\n\tSimilarity:",item[1])

Okta 20--Software-Engineer-Adaptive-Authentication-(Senior_Staff_Principal)-|-Okta.Txt 
	Index: 821 
	Similarity: 0.7168875
Uber 560--Machine-Learning-Systems-Engineer--Self-Driving---San-Francisco--Ca---Uber---Uber.Txt 
	Index: 106 
	Similarity: 0.641787
Affirm Software Engineer - Machine Learning 
	Index: 1339 
	Similarity: 0.639349
Uber 562--Machine-Teaching---Senior-Engineering-Manager--Self-Driving---San-Francisco--Ca---Uber---Uber.Txt 
	Index: 327 
	Similarity: 0.62554973
Affirm Lead Software Engineer - Machine Learning 
	Index: 1313 
	Similarity: 0.6233222


[(0, 6.012634075735337), (1, -3.408114302991006), (2, -2.417287904619967), (3, 1.0195485697266713), (4, -1.0686142673900716), (5, 2.6050128738547214), (6, 2.7048850235510256), (7, -1.7988146467834711), (8, -2.4811391469049697), (9, 1.2154228439409116), (10, 0.5190406650068238), (11, -0.3943437859189026), (12, -1.1218410387980715), (13, -4.501899742560999), (14, 1.1940177042440234), (15, -0.05755995798846737), (16, 2.454958227047167), (17, -2.6672018096263184), (18, 0.9444301178962926), (19, -0.8607403737446442), (20, 1.154947386201633), (21, -0.30240543447435847), (22, -0.2362365533301101), (23, -1.090824757900462), (24, -0.7799754294244103), (25, -0.06407095057498566), (26, -0.10672732358353763), (27, -0.06596671636183801), (28, -0.2568900948416098), (29, -0.007495077343689632), (30, -0.1866381167981653), (31, -0.049809701691324385), (32, 0.2938932006601585), (33, 0.499757550389667), (34, 0.5153361685569829), (35, 0.8606882716296469), (36, -0.6087008971227313), (37, 0.8318344173482317

In [29]:
print(documents[174])

role uber global social medium team leader vision strategy global social medium channel campaign work uber brand marketing editorial communication team cultural relevancy immediacy social channel social voice tone global social content social channel strategy process strategy resource social marketing team inspire innovation creativity global social channel social campaign drive engagement effective storytelling concept idea word brand narrative social calendar central marketing calendar evocative content brand objective social channel evolve experiment brand voice brand style guideline work strategy web campaign team mentor social medium team resource sharing high bar social execution channel city work international team sophisticated strategy audience international social medium channel leader manager multi channel content creation ideation inclass editorial skill track record young strategist writer editor marketing manager communication skill depth expertise online social medium pr

In [30]:
# Word similarities in a high-scored document:
print([word for word in documents[821].split() if word in doc])

['software', 'engineer', 'large', 'scale', 'critical', 'software', 'fast', 'great', 'engineering', 'team', 'design', 'enterprise', 'software', 'elite', 'team', 'weekly', 'release', 'cycle', 'individual', 'ownership', 'great', 'thing', 'engineer', 'new', 'project', 'technology', 'cloud', 'authentication', 'team', 'edge', 'risk', 'adaptive', 'authentication', 'policy', 'software', 'engineer', 'large', 'scale', 'system', 'machine', 'learning', 'policy', 'business', 'problem', 'complex', 'system', 'data', 'authentication', 'policy', 'factor', 'authentication', 'security', 'analyze', 'requirement', 'product', 'management', 'work', 'engineering', 'management', 'engineering', 'effort', 'design', 'test', 'driven', 'development', 'design', 'code', 'review', 'product', 'management', 'team', 'software', 'development', 'experience', 'object', 'language', 'reliable', 'critical', 'software', 'excellent', 'grasp', 'software', 'engineering', 'principle', 'authentication', 'authentication', 'policy', '

In [31]:
# Word similarities in a low-scored document
print([word for word in documents[174].split() if word in doc])

['team', 'leader', 'work', 'team', 'team', 'drive', 'work', 'team', 'team', 'work', 'team', 'leader', 'multi', 'skill', 'skill']


This seems to work pretty well, but we lose all of the structure around the skill phrases, because we just project them into TF-IDF space.

This may work as an MVP solution, that we can hook up to the application.

For entity resolution, maybe we can look at trigram/bigram/unigram embeddings from these words, as trained on the wikipedia corpus. Then:
1. cluster the objects by their embeddings. Set K to be pretty high (the number of distinct skills we want represented -- perhaps 1000). In the near term, maybe we manually prune these clusters -- removing "junk" terms that we know as humans don't qualify as skills
2. Label each skill phrase by its cluster ID
3. Canonicalize the skill phrases by tagging them with a cluster ID. Input is a skill phrase, output is a genericized cluster ID that will be the same for any skill phrase that _is used in a similar context_ as the 

# try with a resume

In [85]:
with open('sample_resume.txt', 'r') as infile:
    resume = infile.read()
doc = parse_input_docs(resume)
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space

In [86]:
sims = index[vec_lsi]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
for item in sims_sorted[:5]:
    print(titles[item[0]],"\n\tIndex:",item[0],"\n\tSimilarity:",item[1])

Salesforce 62--Lead-Data-Engineer---Heroku.Txt 
	Index: 1163 
	Similarity: 0.6560252
Uber 96--Workplace-Program-Manager--Data-Scientist---San-Francisco--Ca---Uber---Uber.Txt 
	Index: 177 
	Similarity: 0.6259361
Square Data Scientist, Commerce Platform 
	Index: 692 
	Similarity: 0.62403953
Salesforce 142--Sr.-Data-Science-Engineer.Txt 
	Index: 1264 
	Similarity: 0.60767066
Salesforce 164--Summer-2018-Intern-Data-Visualization-Engineer-(Infrastructure-Data-Science).Txt 
	Index: 977 
	Similarity: 0.60026777


In [32]:
# Load Google's pre-trained Word2Vec model.
word2vec = gensim.models.KeyedVectors.load_word2vec_format('~/Downloads/GoogleNews-vectors-negative300.bin', binary=True)  

In [92]:
word2vec.wv['product']  # numpy vector of a word

array([-6.15234375e-02,  9.52148438e-02,  1.33789062e-01,  6.49414062e-02,
       -1.20117188e-01,  9.86328125e-02,  2.85156250e-01, -1.21582031e-01,
        1.73828125e-01,  3.85742188e-02, -1.50390625e-01, -1.67968750e-01,
       -4.46777344e-02, -1.55273438e-01, -1.67968750e-01, -2.70996094e-02,
        2.48046875e-01,  6.88476562e-02, -1.23046875e-01,  1.83105469e-02,
       -2.14843750e-02,  7.86132812e-02, -1.99218750e-01,  7.37304688e-02,
        4.63867188e-02,  6.17675781e-02, -7.41577148e-03,  1.62109375e-01,
        1.48437500e-01,  4.71191406e-02, -1.28906250e-01, -5.22460938e-02,
       -1.85546875e-02, -1.22680664e-02,  8.34960938e-02,  2.30407715e-03,
        4.17480469e-02, -3.92578125e-01,  1.54296875e-01, -1.18652344e-01,
       -1.07421875e-01,  3.01513672e-02, -1.67236328e-02, -4.76074219e-02,
       -3.61328125e-01, -1.70898438e-01, -8.49609375e-02,  8.59375000e-02,
        4.49218750e-02,  5.66406250e-02,  8.88671875e-02, -6.73828125e-02,
       -1.00585938e-01, -

In [36]:
word2vec.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674735069275),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.549946129322052),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

## Phrase co-location

As a good preprocessing step. 
https://radimrehurek.com/gensim/models/phrases.html

In [37]:
phrases = gensim.models.Phrases(' '.join(parsed_dataset))

In [38]:
bigram = gensim.models.phrases.Phraser(phrases)

In [39]:
sent = ['machine','learning','is','so','hot','right','now']
print(bigram[sent])

['machine', 'learning', 'is', 'so', 'hot', 'right', 'now']


In [90]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

## SpaCy
From [this example on Github](https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb)

Installation had some oddities; run the following:
```bash
$ pip install -U spacy
$ python -m spacy download en
```

I tried first without preprocessed "skill phrase" text as the corpus. Then, I ran back through with the "skill phrase" text. Both types of models and text output docs are saved in this directory.

In [91]:
import spacy
import pandas as pd
import itertools as it
import en_core_web_sm

import spacy
nlp = spacy.load('en')

In [92]:
parsed_review = nlp(job_str)
print(parsed_review)

Data Scientist/Machine Learning Engineer, Adaptive Authentication
Position Description:

This is an opportunity to join our fast-growing Adaptive Authentication team to develop cutting-edge risk-based adaptive authentication policies. We are looking for a Data Scientist/Machine Learning Engineer to build large-scale distributed systems while using machine learning to solve business problems. The ideal candidate has experience building models from complex systems, developing enterprise-grade software in an object-oriented language, and experience or knowledge in security, authentication or identity.

Our elite team is fast, innovative and flexible; with a weekly release cycle and individual ownership we expect great things from our engineering and reward them with stimulating new projects and emerging technologies.


Job Duties and Responsibilities:

• Build and own forecasting models that identify risk associated with authentication such as anomalous activity in the cloud
• Build Machi

In [43]:
# for num, sentence in enumerate(parsed_review.sents):
#     print('Sentence {}:'.format(num + 1))
#     print(sentence)
#     print()

In [44]:
# token_text = [token.orth_ for token in parsed_review]
# token_pos = [token.pos_ for token in parsed_review]

# pd.DataFrame(list(zip(token_text, token_pos)),
#              columns=['token_text', 'part_of_speech'])

In [45]:
# token_lemma = [token.lemma_ for token in parsed_review]
# token_shape = [token.shape_ for token in parsed_review]

# pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
#              columns=['token_text', 'token_lemma', 'token_shape'])

In [46]:
# token_entity_type = [token.ent_type_ for token in parsed_review]
# token_entity_iob = [token.ent_iob_ for token in parsed_review]

# pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
#              columns=['token_text', 'entity_type', 'inside_outside_begin'])

In [47]:

# token_attributes = [(token.orth_,
#                      token.prob,
#                      token.is_stop,
#                      token.is_punct,
#                      token.is_space,
#                      token.like_num,
#                      token.is_oov)
#                     for token in parsed_review]

# df = pd.DataFrame(token_attributes,
#                   columns=['text',
#                            'log_probability',
#                            'stop?',
#                            'punctuation?',
#                            'whitespace?',
#                            'number?',
#                            'out of vocab.?'])

# df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
#                                        .applymap(lambda x: u'Yes' if x else u''))
                                               
# df

In [93]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

1. Segment text of complete reviews into sentences & normalize text
2. First-order phrase modeling $\rightarrow$ apply first-order phrase 3. model to transform sentences
4. Second-order phrase modeling $\rightarrow$ apply second-order 5. phrase model to transform sentences
6. Apply text normalization and second-order phrase model to text of complete reviews

In [94]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    for review in filename:
        yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [95]:
%%time

import codecs
# This is time consuming; make the if statement True to run
if 0 == 0:
    with codecs.open('spacy_parsed_jobs_PARSED.txt', 'w', encoding='utf_8') as f:
        for sentence in parsed_dataset:
            f.write(sentence + '\n')

CPU times: user 6.71 ms, sys: 2.98 ms, total: 9.69 ms
Wall time: 8.34 ms


In [96]:
unigram_sentences = LineSentence('spacy_parsed_jobs_PARSED.txt')

In [97]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

uber indirect tax manager  state tax team role domestic tax function amazing opportunity high growth environment immediate impact job combination people skill excellent attention superb organizational skill primary indirect tax manager various indirect tax compliance related matter preparation or review annual tax return sale tax gross receipt indirect tax multiple jurisdiction assistance property tax compliance preparation or review property tax return payment appeal reconciliation or review tax account quarterly basis assistance state local business registration as management state local indirect tax audit finance legalit team reporting compliance process team finance operation indirect tax impact new business idea research communicate analysis risk technical memorandum process special project assigned train team member special project assigned ba b degree accounting finance year tax experience multi state environment solid sale property tax concept law strong analytical problem sk

In [98]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 0:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save('spacy_bigram_model_all_PARSED')
    
# load the finished model from disk
bigram_model = Phrases.load('spacy_bigram_model_all_PARSED')

CPU times: user 720 ms, sys: 34 ms, total: 754 ms
Wall time: 760 ms


In [99]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open('spacy_bigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1.15 s, sys: 12.1 ms, total: 1.16 s
Wall time: 1.18 s


In [100]:
bigram_sentences = LineSentence('spacy_bigram_sentences_PARSED.txt')

In [101]:
for bigram_sentence in it.islice(bigram_sentences, 240, 250):
    print(u' '.join(bigram_sentence))
    print(u'')

head business egypt high growth company shape plan mind share broad usage uber mobile app hundred_million consumer region responsibility key market segment necessary business relationship level partner egyptâ middle_east accelerate growth uber rider 2 supply transport uber platform partnership uber profitability uber product team san_francisco uber service address issue concern request field excellent ability business product opportunity demonstrated portfolio_accomplishment high value partnership important company region internet mobile space keen sense responsibility speedy execution initiative individual_contributor deal lead manager engagement key member uber business development team overall market plan uber partnership roadmap strategic business development plan target market egyptâ line overall uber strategic_direction uber top partnership opportunity egypt key internal_stakeholder e.g. product engineering operation specific prospect partner specific value_proposition partner bu

In [102]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 0:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save('spacy_trigram_model_all_PARSED')
    
# load the finished model from disk
trigram_model = Phrases.load('spacy_trigram_model_all_PARSED')

CPU times: user 674 ms, sys: 20 ms, total: 694 ms
Wall time: 698 ms


In [103]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open('spacy_trigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 916 ms, sys: 6.2 ms, total: 922 ms
Wall time: 926 ms


In [104]:
trigram_sentences = LineSentence('spacy_trigram_sentences_PARSED.txt')

In [105]:
for trigram_sentence in it.islice(trigram_sentences, 240, 250):
    print(u' '.join(trigram_sentence))
    print(u'')

head business egypt high_growth company shape plan mind share broad usage uber mobile_app hundred_million consumer region responsibility key market segment necessary business relationship level partner egyptâ middle_east accelerate growth uber rider 2 supply transport uber platform partnership uber profitability uber product team san_francisco uber service address issue concern request field excellent ability business product opportunity demonstrated portfolio_accomplishment high value partnership important company region internet mobile space keen sense responsibility speedy execution initiative individual_contributor deal lead manager engagement key_member uber business development team overall market plan uber partnership roadmap strategic business development plan target market egyptâ line overall uber strategic_direction uber top partnership opportunity egypt key_internal_stakeholder e.g. product engineering operation specific prospect partner specific value_proposition partner bu

### Final processing

In [106]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open('spacy_trigram_transformed_reviews_all_PARSED.txt', 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(dataset.data),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in stopwords]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2min 41s, sys: 33.2 s, total: 3min 15s
Wall time: 1min 52s


In [107]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(dataset.data), 11, 12):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with codecs.open('spacy_trigram_transformed_reviews_all.txt', encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original:

Uber 176--Data-Scientist---Uber-Freight---San-Francisco--Ca---Uber---Uber.Txt





we're looking for experienced, passionate data scientists to work on our team. specifically, we are looking for someone to build the pricing engine that will power uber freight.

theÂ marketplace dynamics group is all about the intersection of market signals and business strategy. its mission is to ensure that uber freight has the data, processes, and tools to build and maintain a healthy marketplace. that means building quoting tools so that our operations teams can bid and sell with confidence; that means building data structures to track the buy-offer-sell spreads of our freight; that means building a dynamically priced marketplace to ensure that our drivers are the most fairly paid in the industry.

you'll be partnering with uber's product and engineering teams to drive actionable data insights and solutions from some of the world's most interesting datasets. our data scientists work on ev

In [108]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [109]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 0 == 0:

    trigram_reviews = LineSentence('spacy_trigram_sentences_PARSED.txt')

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save('spacy_trigram_dict_all.dict')
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load('spacy_trigram_dict_all.dict')

CPU times: user 233 ms, sys: 4.03 ms, total: 237 ms
Wall time: 263 ms


In [110]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [111]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 0 == 0:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize('spacy_trigram_bow_corpus_all.mm',
                       trigram_bow_generator('spacy_trigram_sentences_PARSED.txt'))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus('spacy_trigram_bow_corpus_all.mm')

CPU times: user 317 ms, sys: 7.13 ms, total: 324 ms
Wall time: 327 ms


In [112]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 0 == 0:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lsi = gensim.models.LsiModel(trigram_bow_corpus, 
                                     id2word=trigram_dictionary, 
                                     num_topics=500)
    
    lsi.save('spacy_lsi_model_all')
    
# load the finished LDA model from disk
lsi = LsiModel.load('spacy_lsi_model_all')

CPU times: user 1.95 s, sys: 241 ms, total: 2.19 s
Wall time: 1.28 s


In [136]:
def explore_topic(topic_number, topn=10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
#     print(u'{:20} {}'.format(u'term', u'frequency') + u'')

    for term, frequency in lsi.show_topic(topic_number, topn=10):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [139]:
for i in range(600):
    print("\n\nTopic %s" % str(i+1))
    explore_topic(topic_number=i)



Topic 1
sale                 0.259
marketing            0.245
partner              0.207
solution             0.178
strategy             0.176
project              0.172
salesforce           0.168
program              0.165
technical            0.153
design               0.153


Topic 2
sale                 0.544
marketing            0.457
security             -0.297
¢                   -0.183
design               -0.149
infrastructure       -0.149
technical            -0.142
engineering          -0.137
strategy             0.137
campaign             0.111


Topic 3
sale                 0.594
marketing            -0.350
uber                 -0.304
salesforce           0.189
solution             0.165
security             0.159
¢                   0.146
campaign             -0.144
strategy             -0.126
cloud                0.121


Topic 4
¢                   0.877
marketing            0.179
security             -0.175
design               -0.170
program              0.152
sal

api                  -0.148
high                 0.138
great                0.126
excellent            0.124
multiple             0.122
retail               0.109
accessibility        0.107
map                  0.104
channel              -0.103
feedback             -0.102


Topic 154
excellent            -0.150
competitive          0.135
client               0.125
identity             0.122
expert               0.121
opportunity          -0.116
stakeholder          0.115
merchant             -0.110
consumer             -0.108
social_medium        0.107


Topic 155
oracle               -0.159
api                  0.130
key                  0.120
senior               -0.120
software_engineering -0.118
competitive          -0.116
revenue              0.114
finance              0.114
driver               0.102
group                0.093


Topic 156
future               -0.167
database             0.160
report               -0.129
model                -0.127
challenge            -0.126
card

day                  0.098
investigation        0.096
track_record         -0.091
scalable             0.091
facility             -0.090


Topic 281
competition          -0.120
track_record         0.113
world_class          -0.108
digital              0.102
technique            0.100
android              -0.096
communication_skill  0.094
background           -0.091
act                  0.089
community_operation  0.088


Topic 282
someone              -0.111
talent               0.106
language             -0.106
life                 -0.102
approach             0.098
world_class          0.097
car                  -0.094
capability           0.094
cross_functional     -0.092
priority             0.090


Topic 283
world_class          0.106
android              -0.103
smb                  -0.100
vendor               0.100
salesforce_.com      0.099
understanding        0.098
app                  -0.098
lifecycle            0.097
concept              -0.095
facility             -0.090


T

human                -0.072


Topic 417
data_scientist       0.117
apps                 0.098
recruiter            0.097
python               -0.095
expense              0.088
bachelor_degree      -0.084
internship           0.083
implement            -0.081
someone              0.081
custom               0.076


Topic 418
track                0.109
money                -0.089
competition          0.085
group                -0.083
degree               0.081
retail               0.081
method               0.079
consumer             0.078
recommendation       -0.075
visualization        0.074


Topic 419
deep_learning        0.110
launch               -0.109
hand_on              0.105
seller               0.096
app                  0.092
real_time            -0.091
open_source          0.087
san_francisco        -0.083
equivalent_experience -0.078
architect            -0.076


Topic 420
lifecycle            0.102
general              0.093
communicate          0.087
modern               